# Install

In [1]:
!pip install -q dwave-ocean-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.7/103.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00


In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from google.colab import files
import pickle

In [3]:
from collections import defaultdict
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector as inspector
from dimod import ConstrainedQuadraticModel, CQM, SampleSet
from dwave.system import LeapHybridCQMSampler
from dimod.vartypes import Vartype
from dimod import Binary, quicksum

In [24]:
from google.colab import userdata

# Import Alpha, Beta and Theta data

## Alpha data

In [4]:
alpha_data = files.upload()
alpha_file_name = list(alpha_data.keys())[0]
alpha_file_name

Saving Vision_21_alpha.txt to Vision_21_alpha.txt


'Vision_21_alpha.txt'

In [5]:
alpha = dict()
with open(alpha_file_name) as f:
    lines = f.readlines()
    for line in lines:
      a,b,c = line.replace('\n','').split(' ')
      a = int(a)
      b = int(b)
      c = int(c)
      alpha[(a-1,b-1)] = c
#alpha

## Beta data

In [6]:
beta_data = files.upload()
beta_file_name = list(beta_data.keys())[0]
beta_file_name

Saving Vision_21_beta.txt to Vision_21_beta.txt


'Vision_21_beta.txt'

In [7]:
n_students = 0
with open(beta_file_name) as f:
    lines = f.readlines()
    first_line = lines[0].split(' ')
    #print(first_line)
    n_features = len(first_line)
    #print(n_features)
    beta = [[] for x in range(n_features)]
    for line in lines:
      n_students +=1
      line = line.replace('\n','').split(' ')
      for i in range(n_features):
        beta[i].append(int(line[i]))

In [8]:
for i in range(n_students):
  for j in range(n_students):
    if i==j:
      alpha[(i,j)] = 0

In [ ]:
#alpha

## Theta data

In [9]:
theta_data = files.upload()
theta_file_name = list(theta_data.keys())[0]
theta_file_name

Saving Vision_21_theta.txt to Vision_21_theta.txt


'Vision_21_theta.txt'

In [10]:
theta = []
with open(theta_file_name) as f:
    lines = f.readlines()
    for line in lines:
      theta.append(int(line.replace('\n','')))
theta

[3, 3, 4, 8]

In [11]:
beta_max = [theta[x] for x in range(n_features)]
tao_min = theta[-2]
tao_max = theta[-1]

## Check problem inputs

In [12]:
n_groups = 3

In [13]:
print(f'Arrange classroom of {n_students} students, into {n_groups} groups of min group \
size: {tao_min} and max group size: {tao_max}')

Arrange classroom of 21 students, into 3 groups of min group size: 4 and max group size: 8


# DWAVE Initializing

In [25]:
endpoint = 'https://cloud.dwavesys.com/sapi'
token = userdata.get('dwave_leap')

In [26]:
N,C = range(n_students),range(n_groups)

# Creating Model

In [27]:
cqm = ConstrainedQuadraticModel()

## Creating the variables

In [28]:
x = {(i, c): Binary(f'x{i}_{c}') for i in range(n_students) for c in range(n_groups)}

## Creating the objective function

In [29]:
objective = -1*quicksum(x[i,c]*x[j,c]*alpha.get((i,j),0) for i in N for j in N for c in C)
cqm.set_objective(objective)

## Creating the constraints

In [30]:
# one student per group

for i in N:
  cqm.add_constraint( quicksum(x[i,c] for c in C) == 1 )

In [31]:
# minimum students per group >= tao_min
# Maximum students per group <= tao_max

for c in C:
  cqm.add_constraint( quicksum(x[i,c] for i in N) >= tao_min )
  cqm.add_constraint( quicksum(x[i,c] for i in N) <= tao_max )

In [32]:
# Beta homegeinity constraints
group_combs = [(c,cp) for c in C for cp in C if c!=cp]

In [33]:
for c1,c2 in group_combs:
  for f in range(n_features):
    bf = beta[f]
    B = beta_max[f]
    cqm.add_constraint( quicksum(x[i,c1]*bf[i] for i in N) - quicksum(x[i,c2]*bf[i] for i in N) <= B )

## Run the model

In [34]:
counter = 1
while True:
  try:
    cqm_sampler = LeapHybridCQMSampler(endpoint=endpoint, token=token)
  except:
    if counter <= 5:
      print(f"{counter} -  Problem finding embedding trying it once again...")
      counter += 1
      continue
    else:
      raise Exception(f"Imposible to find an embedding after {counter} tries")
  break

In [35]:
sampleset = cqm_sampler.sample_cqm(cqm,label = f'CGFP_min_sum_{n_students}')

In [36]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

In [45]:
not_feasible_sampleset = sampleset.filter(lambda row: not row.is_feasible)

In [37]:
best = feasible_sampleset.first

In [38]:
best_solution = best.sample

# Verify constraints

In [39]:
def verify_constraints(sample,print_all = False,verbose = False):

  violated = 0

  for i in N:
    just_one = [sample[f'x{i}_{c}'] for c in C]
    if sum(just_one) != 1:
      violated += 1
      if verbose:
        print(f'Individual {i} with {sum(just_one)} assignments')
    if verbose:
      if print_all:
        print(f'Individual {i} with {sum(just_one)} assignments')

  GROUPS = [ []  for g in C]
  for var in sample:
    if 'eta' in var:
      continue
    if sample[var]==1:
      st,g = var[1:].split('_')
      st = int(st)
      g = int(g)
      GROUPS[g].append(st)
      # print(f'student_{st} belongs to group {g}')

  for i,g in enumerate(GROUPS):
    spg = len(g)
    valid = spg >= tao_min and spg <= tao_max
    if not valid:
      violated += 1
      if verbose:
        print(f'* number of members of group_{i}: ({tao_min} <= {len(g)} <= {tao_max}), valid = {valid}')
    if verbose:
      if print_all:
        print(f'* number of members of group_{i}: ({tao_min} <= {len(g)} <= {tao_max}), valid = {valid}')
  for c1,c2 in group_combs:

    G1 = GROUPS[c1]
    G2 = GROUPS[c2]
    for f in range(n_features):
      bf = beta[f]
      B = beta_max[f]
      val1 = sum([bf[x] for x in G1])
      val2 = sum([bf[x] for x in G2])
      subs = val1 - val2
      valid = subs >= -B and subs <= B

      if not valid:
        violated += 1
        if verbose:
          print(f'(g{c1},g{c2}) feature = {f},| {-B} <= {subs} <= {B}| valid = {valid} ')
      if verbose:
        if print_all:
          print(f'(g{c1},g{c2}) feature = {f},| {-B} <= {subs} <= {B}| valid = {valid} ')
  return violated

In [40]:
GROUPS = [ []  for g in C]
for var in best_solution:
  if best_solution[var]==1:
    st,g = var[1:].split('_')
    st = int(st)
    g = int(g)
    GROUPS[g].append(st)
    print(f'student_{st} belongs to group {g}')

student_0 belongs to group 1
student_10 belongs to group 2
student_11 belongs to group 0
student_12 belongs to group 2
student_13 belongs to group 0
student_14 belongs to group 2
student_15 belongs to group 0
student_16 belongs to group 1
student_17 belongs to group 0
student_18 belongs to group 0
student_19 belongs to group 2
student_1 belongs to group 2
student_20 belongs to group 2
student_2 belongs to group 2
student_3 belongs to group 1
student_4 belongs to group 0
student_5 belongs to group 2
student_6 belongs to group 1
student_7 belongs to group 1
student_8 belongs to group 1
student_9 belongs to group 1


In [41]:
for i,g in enumerate(GROUPS):
  spg = len(g)
  valid = spg >= tao_min and spg <= tao_max
  print(f'* number of members of group_{i}: ({tao_min} <= {len(g)} <= {tao_max}), valid = {valid}')

* number of members of group_0: (4 <= 6 <= 8), valid = True
* number of members of group_1: (4 <= 7 <= 8), valid = True
* number of members of group_2: (4 <= 8 <= 8), valid = True


In [42]:
for c1,c2 in group_combs:
  #print((c1,c2),GROUPS[c1],GROUPS[c2])
  G1 = GROUPS[c1]
  G2 = GROUPS[c2]
  for f in range(n_features):
    bf = beta[f]
    B = beta_max[f]
    #print(bf)
    val1 = sum([bf[x] for x in G1])
    val2 = sum([bf[x] for x in G2])
    subs = val1 - val2
    condition = subs >= -B and subs <= B

    print(f'(g{c1},g{c2}) feature = {f},| {-B} <= {subs} <= {B} | valid? = {condition}')

(g0,g1) feature = 0,| -3 <= -3 <= 3 | valid? = True
(g0,g1) feature = 1,| -3 <= 1 <= 3 | valid? = True
(g0,g2) feature = 0,| -3 <= -3 <= 3 | valid? = True
(g0,g2) feature = 1,| -3 <= -2 <= 3 | valid? = True
(g1,g0) feature = 0,| -3 <= 3 <= 3 | valid? = True
(g1,g0) feature = 1,| -3 <= -1 <= 3 | valid? = True
(g1,g2) feature = 0,| -3 <= 0 <= 3 | valid? = True
(g1,g2) feature = 1,| -3 <= -3 <= 3 | valid? = True
(g2,g0) feature = 0,| -3 <= 3 <= 3 | valid? = True
(g2,g0) feature = 1,| -3 <= 2 <= 3 | valid? = True
(g2,g1) feature = 0,| -3 <= 0 <= 3 | valid? = True
(g2,g1) feature = 1,| -3 <= 3 <= 3 | valid? = True


In [43]:
z_obj = 0
groups_happiness = []
for group in GROUPS:
  happy = 0
  for s1 in group:
    for s2 in group:
      if s1 != s2:
        happy += alpha.get((s1,s2),0)
        z_obj += alpha.get((s1,s2),0)
  groups_happiness.append(happy)

In [44]:
for i,g in enumerate(GROUPS):
  print(f'group {i}, hapiness = {groups_happiness[i]}')
print(f'Total Happiness = {z_obj}')

group 0, hapiness = 29
group 1, hapiness = 9
group 2, hapiness = 22
Total Happiness = 60


# Verify all not feasible constraints

In [46]:
for sample in not_feasible_sampleset:
  print(verify_constraints(sample))
  print("-"*120)

26
------------------------------------------------------------------------------------------------------------------------
22
------------------------------------------------------------------------------------------------------------------------
15
------------------------------------------------------------------------------------------------------------------------
13
------------------------------------------------------------------------------------------------------------------------
12
------------------------------------------------------------------------------------------------------------------------
5
------------------------------------------------------------------------------------------------------------------------
9
------------------------------------------------------------------------------------------------------------------------
8
------------------------------------------------------------------------------------------------------------------------
6
---------

# Extract solution txt

In [47]:
x_dict = dict()
for var in best_solution:
  if 'eta' in var:
    continue
  if best_solution[var]==1:
    st,g = var[1:].split('_')
    st = int(st) + 1
    g = int(g) + 1
    x_dict[st] = g
x_dict = sorted(x_dict.items(), key=lambda x:x[0])

In [48]:
x_dict

[(1, 2),
 (2, 3),
 (3, 3),
 (4, 2),
 (5, 1),
 (6, 3),
 (7, 2),
 (8, 2),
 (9, 2),
 (10, 2),
 (11, 3),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 3),
 (16, 1),
 (17, 2),
 (18, 1),
 (19, 1),
 (20, 3),
 (21, 3)]

In [49]:
groups = [[] for x in range(n_groups)]
zetas = [0 for x in range(n_groups)]
for x,group in x_dict:
  groups[group-1].append(x)


for i,G in enumerate(groups):
  happy = 0
  for s1 in G:
    for s2 in G:
      if s1 != s2:
        happy += alpha.get((s1-1,s2-1),0)
  print(f"group {i}, Z = {happy}")

group 0, Z = 29
group 1, Z = 9
group 2, Z = 22


# Save best solution txt

In [ ]:
#one solution only

# f = open("x_min_max_100.txt", "w")
# for i,j in x_dict:
#   f.write(f"{i} {j}\n")
# f.write(f"{Z}")
# f.close()

# Extract Multiple Solution

In [50]:
def generate_all_solutions_dict(sampleset):

  solutions = []

  for sample in sampleset:
    sample = dict(sample)
    x_dict = dict()
    for var in sample:
      if 'eta' in var:
        continue
      if sample[var]==1:
        st,g = var[1:].split('_')
        st = int(st) + 1
        g = int(g) + 1
        x_dict[st] = g
    x_dict = sorted(x_dict.items(), key=lambda x:x[0])
    solutions.append(x_dict)
  return solutions

## Save multiple feasible solutions

In [51]:
all_feasible_x = generate_all_solutions_dict(feasible_sampleset)

In [53]:
# multiple solutions

f = open(f"x_min_sum_{n_students}_multiple_feasible.txt", "w")
f.write(f"{len(all_feasible_x)}\n")
for x_d in all_feasible_x:
  for i,j in x_d:
    f.write(f"{i} {j}\n")
  # f.write("---\n")
f.close()

In [54]:
# save it as a pickle file

with open(f"x_min_sum_{n_students}_multiple_feasible.pkl", 'wb') as f:
  pickle.dump(all_feasible_x, f)

## Save not feasible solutions

In [55]:
all_not_feasible_x = generate_all_solutions_dict(not_feasible_sampleset)

In [56]:
# multiple solutions

f = open(f"x_min_sum_{n_students}_multiple_not_feasible.txt", "w")
f.write(f"{len(all_not_feasible_x)}\n")
for x_d in all_not_feasible_x:
  for i,j in x_d:
    f.write(f"{i} {j}\n")
  # f.write("---\n")
f.close()

In [57]:
# save it as a pickle file

with open(f"x_min_sum_{n_students}_multiple_not_feasible.pkl", 'wb') as f:
  pickle.dump(all_not_feasible_x, f)

# Extract solution in txt

In [ ]:
# x_dict = dict()
# for var in best_solution:
#   if best_solution[var]==1:
#     st,g = var[1:].split('_')
#     st = int(st) + 1
#     g = int(g) + 1
#     x_dict[st] = g
# x_dict = sorted(x_dict.items(), key=lambda x:x[0])

In [ ]:
# x_dict

In [ ]:
# z = -1*best.energy
# z

639.0

In [ ]:
# f = open(f"x_min_sum_{n_students}.txt", "w")
# for i,j in x_dict:
#   f.write(f"{i} {j}\n")
# f.write(str(int(z)))
# f.close()